# COVID - 19

Questions:



Datasource: Google Cloud - Public Datasets, Covid19_usafacts

In [ ]:
import pandas as pd

In [5]:
#create dataframe from
df = pd.read_csv(r'C:\Users\Chunna\Documents\Jupterlab\Covid\confirmed_cases.csv')
print(df.head())

   county_fips_code       county_name state  state_fips_code  _2020_01_22  \
0              1003   Baldwin County     AL                1            0   
1              1015   Calhoun County     AL                1            0   
2              1017  Chambers County     AL                1            0   
3              1031    Coffee County     AL                1            0   
4              1051    Elmore County     AL                1            0   

   _2020_01_23  _2020_01_24  _2020_01_25  _2020_01_26  _2020_01_27  ...  \
0            0            0            0            0            0  ...   
1            0            0            0            0            0  ...   
2            0            0            0            0            0  ...   
3            0            0            0            0            0  ...   
4            0            0            0            0            0  ...   

   _2021_09_12  _2021_09_13  _2021_09_14  _2021_09_15  _2021_09_16  \
0        34726  

In [15]:
#manipulate data: remove columns & unpivot
df_drop = df.drop(columns=['county_fips_code','county_name','state_fips_code'])
df_unpivot = df_drop.melt(id_vars=['state'], var_name='date', value_name='cases')
df_unpivot

0         0.000
1         0.000
2         0.000
3         0.000
4         0.000
          ...  
594379    0.301
594380    0.663
594381    1.228
594382    0.200
594383    0.559
Name: cases, Length: 594384, dtype: float64

In [12]:
df_unpivot.describe()

,cases
count,5.943840e+05
mean,1.407458e+04
std,4.318904e+04
min,0.000000e+00
25%,3.280000e+02
50%,3.116000e+03
75%,1.231800e+04
max,1.381978e+06


## Which top 5 states has the most covid cases?

In [26]:
df_unpivot.groupby('state')[['cases']].sum().sort_values('cases', ascending=False).head(5).round({'cases': 0}).astype(int).reset_index()

,state,cases
0,CA,1129431790
1,TX,761759977
2,FL,708179409
3,NY,613405335
4,IL,356670507
